<a href="https://colab.research.google.com/github/Lasal0425/ML_Coursework_Breast_Cancer_Detection/blob/main/Notebook03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score, mean_squared_error, r2_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree
import joblib

In [4]:
# Load the classification dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Coursework/cancer_classification_dataset.csv')
print("Dataset loaded successfully with shape:", df.shape)
print(df.head())

Dataset loaded successfully with shape: (4024, 16)
   Month_of_Birth   Age     Sex T_Stage N_Stage 6th_Stage  \
0              12  68.0  Female      T1      N1       IIA   
1              12  50.0  Female      T2      N2      IIIA   
2              11  58.0  Female      T3      N3      IIIC   
3               3  58.0  Female      T1      N1       IIA   
4               1  47.0  Female      T2      N1       IIB   

              Differentiated  Grade   A_Stage  Tumor_Size Estrogen_Status  \
0      Poorly differentiated      3  Regional         4.0        Positive   
1  Moderately differentiated      2  Regional        35.0        Positive   
2  Moderately differentiated      2  Regional        63.0        Positive   
3      Poorly differentiated      3  Regional        18.0        Positive   
4      Poorly differentiated      3  Regional        41.0        Positive   

  Progesterone_Status  Regional_Node_Examined  Reginol_Node_Positive  \
0            Positive                    24.0  

In [10]:
print('Sex categories are:', df['Sex'].unique())
print('Sex categories are:', df['T_Stage'].unique())
print('Sex categories are:', df['N_Stage'].unique())
print('Sex categories are:', df['6th_Stage'].unique())
print('Sex categories are:', df['Differentiated'].unique())
print('Sex categories are:', df['A_Stage'].unique())
print('Sex categories are:', df['Grade'].unique())
print('Sex categories are:', df['Estrogen_Status'].unique())
print('Sex categories are:', df['Progesterone_Status'].unique())
print('Sex categories are:', df['Mortality_Status'].unique())

Sex categories are: ['Female' '1']
Sex categories are: ['T1' 'T2' 'T3' 'T4']
Sex categories are: ['N1' 'N2' 'N3']
Sex categories are: ['IIA' 'IIIA' 'IIIC' 'IIB' 'IIIB']
Sex categories are: ['Poorly differentiated' 'Moderately differentiated' 'Well differentiated'
 'Undifferentiated']
Sex categories are: ['Regional' 'Distant']
Sex categories are: [3 2 1 4]
Sex categories are: ['Positive' 'Negative']
Sex categories are: ['Positive' 'Negative']
Sex categories are: ['Alive' 'Dead' 'ALIVE' 'DEAD' 'ALive' 'alive' 'dead']


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Month_of_Birth          4024 non-null   int64  
 1   Age                     4024 non-null   float64
 2   Sex                     4024 non-null   object 
 3   T_Stage                 4024 non-null   object 
 4   N_Stage                 4024 non-null   object 
 5   6th_Stage               4024 non-null   object 
 6   Differentiated          4024 non-null   object 
 7   Grade                   4024 non-null   int64  
 8   A_Stage                 4024 non-null   object 
 9   Tumor_Size              4024 non-null   float64
 10  Estrogen_Status         4024 non-null   object 
 11  Progesterone_Status     4024 non-null   object 
 12  Regional_Node_Examined  4024 non-null   float64
 13  Reginol_Node_Positive   4024 non-null   int64  
 14  Survival_Months         4024 non-null   